In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, RandomizedSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score

from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

import optuna

import lightgbm as lgb
import seaborn as sb
import xgboost as xgb
from catboost import CatBoostRegressor

In [214]:
train_data = pd.read_csv('data/train.csv')
training = pd.read_csv('data/train.csv')
testing = pd.read_csv('data/test.csv')
test_data = pd.read_csv('data/test.csv')

y1 = training["Y1"]
y2 = training["Y2"]

print(train_data)

        time         A         B         C         D         E         F  \
0          0  0.207366 -0.159951 -0.634176 -0.580962 -0.266505  0.060173   
1          1  0.188828 -0.265508  0.042143 -0.550442 -0.132319 -0.185219   
2          2 -0.144261 -0.577142 -0.214634 -0.747391 -0.184255 -0.464831   
3          3  0.208982 -0.310449  0.513708 -0.562868  0.742308 -0.305487   
4          4  0.093320 -0.358156  0.173188 -0.687296 -0.161461 -0.116062   
...      ...       ...       ...       ...       ...       ...       ...   
79995  80000 -0.339802 -0.372094 -0.120952 -0.363438  0.266299 -0.222866   
79996  80001 -0.421921 -0.222554 -0.689585 -0.512266 -0.206562 -0.494450   
79997  80002 -0.467038 -0.282018  0.010707 -0.589594  0.067635 -0.410820   
79998  80003 -0.369710 -0.322384 -0.540326 -0.563206 -0.568736 -0.550024   
79999  80004 -0.304899 -0.047375 -0.932102 -0.229424 -0.474211 -0.336000   

              G         H         I         J         K         L         M  \
0     -0

In [215]:
test_data = test_data.drop(columns=["id","time"])
train_data = train_data.drop(columns=["time","Y1", "Y2",])


In [216]:
for col in train_data:
    train_data[f"{col}_rolling_mean_5"] = train_data[col].rolling(5).mean()
    train_data[f"{col}_rolling_std_5"] = train_data[col].rolling(5).std()
    train_data[f"{col}_diff1"] = train_data[col].diff(1)

In [217]:
print(train_data)

              A         B         C         D         E         F         G  \
0      0.207366 -0.159951 -0.634176 -0.580962 -0.266505  0.060173 -0.475257   
1      0.188828 -0.265508  0.042143 -0.550442 -0.132319 -0.185219  0.028295   
2     -0.144261 -0.577142 -0.214634 -0.747391 -0.184255 -0.464831 -0.085181   
3      0.208982 -0.310449  0.513708 -0.562868  0.742308 -0.305487  0.762246   
4      0.093320 -0.358156  0.173188 -0.687296 -0.161461 -0.116062 -0.245748   
...         ...       ...       ...       ...       ...       ...       ...   
79995 -0.339802 -0.372094 -0.120952 -0.363438  0.266299 -0.222866 -0.000358   
79996 -0.421921 -0.222554 -0.689585 -0.512266 -0.206562 -0.494450 -0.197705   
79997 -0.467038 -0.282018  0.010707 -0.589594  0.067635 -0.410820  0.032695   
79998 -0.369710 -0.322384 -0.540326 -0.563206 -0.568736 -0.550024 -0.339077   
79999 -0.304899 -0.047375 -0.932102 -0.229424 -0.474211 -0.336000 -0.573839   

              H         I         J  ...   K_diff1 

In [218]:
print(train_data.columns.tolist())
print(test_data.columns.tolist())

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'A_rolling_mean_5', 'A_rolling_std_5', 'A_diff1', 'B_rolling_mean_5', 'B_rolling_std_5', 'B_diff1', 'C_rolling_mean_5', 'C_rolling_std_5', 'C_diff1', 'D_rolling_mean_5', 'D_rolling_std_5', 'D_diff1', 'E_rolling_mean_5', 'E_rolling_std_5', 'E_diff1', 'F_rolling_mean_5', 'F_rolling_std_5', 'F_diff1', 'G_rolling_mean_5', 'G_rolling_std_5', 'G_diff1', 'H_rolling_mean_5', 'H_rolling_std_5', 'H_diff1', 'I_rolling_mean_5', 'I_rolling_std_5', 'I_diff1', 'J_rolling_mean_5', 'J_rolling_std_5', 'J_diff1', 'K_rolling_mean_5', 'K_rolling_std_5', 'K_diff1', 'L_rolling_mean_5', 'L_rolling_std_5', 'L_diff1', 'M_rolling_mean_5', 'M_rolling_std_5', 'M_diff1', 'N_rolling_mean_5', 'N_rolling_std_5', 'N_diff1']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']


In [219]:
for col in test_data:
    test_data[f"{col}_rolling_mean_5"] = test_data[col].rolling(5).mean()
    test_data[f"{col}_rolling_std_5"] = test_data[col].rolling(5).std()
    test_data[f"{col}_diff1"] = test_data[col].diff(1)

In [220]:
print(test_data)

              A         B         C         D         E         F         G  \
0     -0.371888 -0.273485  0.538326 -0.377691  1.283159 -0.286957  0.439415   
1     -0.459598 -0.514915 -0.235153 -0.262379  0.044343 -0.429888 -0.059034   
2     -0.381609 -0.265023 -0.629131 -0.186288 -0.146302 -0.357875  0.114223   
3     -0.371423 -0.106279 -1.142702 -0.200429 -0.176918 -0.354048 -0.230199   
4     -0.309393 -0.015144 -1.099371 -0.405841 -0.239068 -0.403306 -0.983134   
...         ...       ...       ...       ...       ...       ...       ...   
15991 -0.453361 -0.196443 -0.050846 -0.447012 -0.077724 -0.592894 -0.161817   
15992 -0.481709 -0.258110 -0.454017 -0.603110  0.040541 -0.508827 -0.062558   
15993 -0.457018 -0.300467  0.403098 -0.555525  0.255084 -0.561159  0.126289   
15994 -0.366164 -0.059930  0.541362 -0.505980  0.351848 -0.450031 -0.220432   
15995 -0.538641 -0.382445  0.113962 -0.498791  0.697350 -0.629678  0.146723   

              H         I         J  ...   K_diff1 

In [221]:
parameters = {"num_leaves": [64],
              "learning_rate": [0.02],
              "n_estimators": [400],
              "max_depth": [9]}

grid_y1 = GridSearchCV(lgb.LGBMRegressor(objective="regression", random_state=1),
                    param_grid=parameters,
                    #cv=TimeSeriesSplit(n_splits=5),
                    scoring="r2",
                    n_jobs=-1)

grid_y1.fit(train_data,y1)

print("best param for Y1", grid_y1.best_params_)
print("best cross val r2", grid_y1.best_score_)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005133 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 56
[LightGBM] [Info] Start training from score -0.004542
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010816 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 56
[LightGBM] [Info] Start training from score -0.004483
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the trai

In [222]:
parameters = {"num_leaves": [60],
              "learning_rate": [0.01],
              "n_estimators": [500],
              "max_depth": [10]}

grid_y2 = GridSearchCV(lgb.LGBMRegressor(objective="regression", random_state=1),
                    param_grid=parameters,
                    cv=TimeSeriesSplit(n_splits=5),
                    scoring="r2",
                    n_jobs=-1)

grid_y2.fit(train_data,y2)

print("best param for Y2", grid_y2.best_params_)
print("best cross val r2", grid_y2.best_score_)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 26668, number of used features: 56
[LightGBM] [Info] Start training from score -0.102749
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003323 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 40001, number of used features: 56
[LightGBM] [Info] Start training from score -0.123157
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003906 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 53334, number of used features: 56
[LightGBM] [Info] Star

In [223]:
model_feature_y1 = lgb.LGBMRegressor(objective="regression",
                                  random_state=1,
                                  **grid_y1.best_params_)
model_feature_y1.fit(train_data, y1)

model_feature_y2 = lgb.LGBMRegressor(objective="regression",
                                  random_state=1,
                                  **grid_y2.best_params_)
model_feature_y2.fit(train_data, y2)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003660 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 56
[LightGBM] [Info] Start training from score -0.002807
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

,boosting_type,'gbdt'
,num_leaves,60
,max_depth,10
,learning_rate,0.01
,n_estimators,500
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [224]:
y1_ft_pred = model_feature_y1.predict(test_data)
y2_ft_pred = model_feature_y2.predict(test_data)

In [226]:
new_ft_preds_6 = pd.DataFrame({"id":testing["id"],
                      "Y1":y1_ft_pred,
                      "Y2":y2_ft_pred})

print(new_ft_preds_6)

          id        Y1        Y2
0          1  0.544603 -0.525416
1          2 -0.229755 -0.555854
2          3 -0.294679 -0.451220
3          4 -0.434897 -0.157349
4          5 -0.905844 -0.186976
...      ...       ...       ...
15991  15992 -0.241065 -0.393539
15992  15993 -0.142898 -0.479969
15993  15994  0.280786 -0.429121
15994  15995  0.485384 -0.431099
15995  15996  0.306670 -0.470235

[15996 rows x 3 columns]


In [227]:
new_ft_preds_6.to_csv('new_ft_preds_6.csv', index=False)